# Cryptography - How to exchange messages secrtly even if everyone is listening?

State of the art ciphers use keys to encrypt and decrypt messages. The key is a number which is agreed upon before the parties exchange their messages. Only knowledge of the chosen key allows to decrypt the encrypted messages. Therefore, it is crucial that the key is kept secret at all moments. This begs the question: **how can two parties agree on a secret key when they must fear that all their communication is intercepted?**

## 1. Diffie-Hellman key exchange

We assume that two parties, Alice and Bob, want to find a common key.

First, Alice and Bob choose two keys. This may happen publicly, since those keys are distinct from the key that is generated in the end.

In [67]:
public1 = 3 # base
public2 = 661889 # modulus

Now, Alice and Bob, choose one private key each.

In [68]:
### Exercise 1 ###
# Define Alice's and Bob's private key to be any number between 0 and 10 of your choice.
alicePrivate = 519932
bobPrivate = 341124

Those keys are to be kept secret at all time: only Alice knows her key, and only Bob knows his key.
So, Alice and Bob encrpyt their respective keys, before they sent it to each other.

In [46]:
def encryptKey(privateKey, publicKey1, publicKey2):
    ### Exercise 2 ###
    # Define the function that encrypts the private key and returns it.
    return publicKey1 ** privateKey % publicKey2

In [69]:
### Exercise 3 ###
# Apply the function encryptKey() to define Alice's and Bob's encrypted keys, and print the result.
aliceEncrypted = encryptKey(alicePrivate, public1, public2)
bobEncrypted = encryptKey(bobPrivate, public1, public2)
print("Alice's encrypted key:", aliceEncrypted)
print("Bob's encrypted key:", bobEncrypted)

Alice's encrypted key: 579498
Bob's encrypted key: 414872


Finally, the common key is defined to be:

In [ ]:
commonKey = public1 ** (alicePrivate * bobPrivate) % public2
print(commonKey)

But Alice and Bob don't know each other's private key, only its encrypted version. However, this is enough to compute the common key.

In [70]:
def computeCommonKey(privateKey, encryptedKey, publicKey2):
    ### Exercise 5 ###
    # Define the function that computes the common key using only the parameters declared in the header of this function.
    return encryptedKey ** privateKey % publicKey2

In [71]:
### Exercise 6 ###
# Compute the common key using only the information that is available to Alice (that means not using bobPrivate).
# Repeat the same but for Bob.
# Print the results and compare them. Do they end up with the same private keys?
commonKey_Alice = computeCommonKey(alicePrivate, bobEncrypted, public2)
commonKey_Bob = computeCommonKey(bobPrivate, aliceEncrypted, public2)
print("Alice computes the following common key:", commonKey_Alice)
print("Bob computes the following common key:", commonKey_Bob)

Alice computes the following common key: 317409
Bob computes the following common key: 317409


## 2. Attacks on the Diffie-Hellman key exchange

Without knowledge of either Alice's or Bob's private key an attacker cannot compute the common key. However, an attacker could try to find Alice's (or Bob's) private key using brute force, that means by trying every possibity.

In [51]:
def decrpytKey(encryptedKey, publicKey1, publicKey2):
    ### Exercise 7 ###
    # Define a function that goes through all possible private keys systematically to find the one that matches the encrypted key.
    for possibleKey in range(0, publicKey2 - 1):
        if encryptKey(possibleKey, publicKey1, publicKey2) == encryptedKey:
            return possibleKey

In [ ]:
### Exercise 8 ###
# Apply the above function to find Alice's private key.
# How hard was it to find Alice's private key using only public information?
alicePrivate_hacked = decrpytKey(aliceEncrypted, public1, public2)
print("Alice's private key has been hacked:", alicePrivate_hacked)

In our example, it took very little effort to find Alice's and Bob's private key using only public information. This is, because our public keys are small. In practice, ``public2`` should be at least 2048 bit long!

### 2.1. How to choose good public keys (mathematical digression)

Before we redefine ``public1`` and ``public2``, there is one more thing to consider. Imagine for a second we had chosen ``public1 = 1``. In this case, the common key generated would be
```
commonKey = public1 ** (alicePrivate * bobPrivate) % public2 = 1
```
no matter what Alice's and Bob's private key are, making it easy for an attacker to find the common key. In our example, ``public1`` and ``public2`` are chosen such that (depending on Alice's and Bob's private key) the common key can attain any value from ``1`` to ``public2 - 1``.

In [24]:
### Exercise 9 ###
# Output all values the common key can attain, and thereby support the above claim.
for i in range(0, public2):
    print(public1 ** i % public2)

1
2
4
8
5
10
9
7
3
6
1


In [25]:
### Exercise 10 ###
# Go through Exercise 9 again, but this time with different choices for public1.
# Find at least one choice for public1 that cannot generate all numbers from 1 to public2 - 1.
public1 = 3
for i in range(0, public2):
    print(public1 ** i % public2)

1
3
9
5
4
1
3
9
5
4
1


Mathematicians say that ``public1`` is a primitive root modulo ``public2``, if the expression
```
public1 ** i % public2
```
attains all values from ``1`` to ``public2 - 1`` as ``i`` ranges through all integers. A fancy mathematical theorem guarantess that such primitive roots exist provided that ``public2`` is a prime number.
What we should take away from this mathematical digression are two things.

(1) ``public2`` should be chosen to be a large prime number, and

(2) ``public1`` should be chosen to be a primitive root modulo ``public2``.

In [53]:
### Exercise 11 (involved) ###

# Execute the Diffie-Hellman key exchange again, but this time with higher values for the public key.
# Also try to attack Alice's private key again.
# How high do you have to choose the public key for the brute force attack to become ineffective?

# Feel free to import the file Library.py which you find for download on the github page.
# There you find a generator for prime numbers and a generator for primitive roots.
# (If you struggle with importing the library, have a look at the table below.)

# You may run into problems when exponentiating such large numbers. Make your code more efficient where possible.
# Hint: Check out the function exponentiationBySquaring() in Library.py.